# Create Neo4J database

In this notebook, the data is transported from the staging area (Mongo DB) to the Neo4j graph database. This process consists of several steps. First, the targeted data model of the graph database is described. Then, the technical preparation steps are performed. After that, the regional areas, the stations and the trains are inserted into the graph database.

## Data model

A neo4j database consists of **nodes** connected by **edges**. Nodes and edges can have labels and attributes.

Our data model is shown in the following figure (circles represent nodes, arrows represent edges).

![databaseDesign](images/databaseDesign.png)

There are three different types of nodes:
1. **State** - Attributes: stateId, name, region (West or East Germany).
1. **County** - Attributes: countyID, gross domestic product (GDP) (per capita), unemploymentRate, population, areaSize.
1. **Station** - Attributes: stationKey, name, latitude, longitude.

Stations are located in counties. Counties are located in states. These relationships are represented as edges labeled **LOCATED_IN**.

In addition, the stations are connected to each other by trains. These relationships are represented as edges labeled **TRAIN**. The train edges have different attributes:
* trainNumber
* trainType (e.g. ICE, IC, EC)
* lag (delay number)
* depTime (departure time)
* arrTime (arrival time)
* duration (arrTime - depTime)
* bordrestaurant (True or False)
* bordbistro (True or False)
* bicycle (can bicycles be taken - True or False)

Connections between stations using TRAIN edges are demonstrated using the fictitious **ICE 100** train.

| Station       | Arrival Time | Departure Time | Notes          |
| ------------- | :----------: | :------------: | ---------------|
| Basel Bad Bf  |              |      18:50     | Bordrestaurant |
| Freiburg Hbf  |    19:20     |      19:22     | Bordrestaurant |
| Karlsruhe Hbf |    20:22     |      20:24     |                |
| Stuttgart Hbf |    21:00     |                |                |

The fictive train would be mapped in the graph database as follows (only the edge attributes are listed).

![exampleTrain](images/exampleTrain.png)

## Preparatory activities

Before transferring the data to the graph database, some preparatory steps have to be done. First of all, some required packages have to be loaded.

In [1]:
from pymongo import MongoClient
import pandas as pd
from py2neo import Graph
from py2neo.bulk import create_nodes, create_relationships
from datetime import timedelta
import html
import datetime

In the next code cells, the connections to the Mongo DB database (**source**) and to the neo4j database (**destination**) are established.

Mongo DB connection:

In [2]:
# connect to database server
client = MongoClient('mongodb://localhost:27017/')

# select database
db = client["deutscheBahn"]

Neo4j connection:

In [3]:
g = Graph("bolt://localhost:7687", 
        name="deutscheBahn",            # database name
        auth=("neo4j", "password"))

During the data transfer, the complete data is loaded into the graph database. To avoid duplicates, all existing data in the neo4j database must be deleted beforehand.  This is done in the next code cell.

In [52]:
g.run("MATCH (n) DETACH DELETE n")

(No data)

## Regional areas

In this section, the regional areas are transferred to the graph database. This includes the nodes **counties**, **states** and the **relationship (edges)** between both.

First, the data is retrieved from the Mongo DB source and stored in Pandas Dataframe `regionalDf`. 

In [53]:
# load data
regionalDf = pd.DataFrame(list(db.counties.find({},{"_id":0})))

# display first rows
regionalDf.head()

,countyId,countyName,gdp,gdpPerCapita,unemploymentRate,population,areaSize,stateId,state,region
0,01001,"Flensburg, kreisfreie Stadt",3733513,41944,8.0,90164,53.02,01,Schleswig-Holstein,West Germany
1,01002,"Kiel, Landeshauptstadt, kreisfreie Stadt",11823188,47723,7.6,246794,118.65,01,Schleswig-Holstein,West Germany
2,01003,"Lübeck, Hansestadt, kreisfreie Stadt",9367966,43219,7.3,216530,214.19,01,Schleswig-Holstein,West Germany
3,01004,"Neumünster, kreisfreie Stadt",3458069,43546,7.9,80196,71.66,01,Schleswig-Holstein,West Germany
4,01051,"Dithmarschen, Landkreis",4606985,34554,5.5,133193,1428.17,01,Schleswig-Holstein,West Germany


Edges can only be created when all associated nodes are created. Therefore the nodes are created first.

The bulk load of the nodes can be done with the `create_nodes(connection, data, labels)` function of the `py2neo.bulk` module. The data passed to the function (parameter 2) must be in a certain format, e.g. a list of dictionaries. Each dictionary represents a node. The keys of the dictionaries are the attributes. 

The **county** nodes are inserted first. We take the columns of the `regionalDf` data frame as node attributes except `stateId`, `state` and `region`, since the connection with the states is realized via edges. The value of the column `countyName` is stored as an attribute with the name `name`. This results that the name of the node being displayed in graph charts.

In [54]:
# remove state oriented informations
countyNodes = regionalDf.drop(columns=["stateId", "state", "region"])

# rename county name
countyNodes = countyNodes.rename(columns={"countyName":"name"})

# create list of nested dictionaries
countyNodes = countyNodes.to_dict("records")

# insert county nodes --> assign label "County"
create_nodes(g.auto(), countyNodes, labels={"County"})

Next, we insert the state nodes. We generate the data from the `regionalDf` data frame, according to the same principle as before. Since this data frame contains one row per county, we need to remove duplicates after selecting the relevant columns.

In [55]:
# only state oriented informations
stateNodes = regionalDf[["stateId", "state", "region"]]

# rename state name
stateNodes = stateNodes.rename(columns={"state":"name"})

# drop duplicates
stateNodes = stateNodes.drop_duplicates()

# create list of nested dictionaries
stateNodes = stateNodes.to_dict("records")

# insert state nodes --> assign label "State"
create_nodes(g.auto(), stateNodes, labels={"State"})

Finally, the edges from the county nodes to the the state nodes are created (label `LOCATED_IN`). This is done with the function `create_relationships(connection, data, label, start_node_properties, end_node_properties)` from the `py2neo.bulk` module. The data passed to the function (parameter 2) must be a list of nested tuples. Each tuple consists of three elements. The first element specifies the key(s) of the starting node. The second element is a dictionary containing the attributes of the edge. The third element specifies the key(s) of the end node.

In [56]:
# only connection columns
countyStateEdges = regionalDf[["countyId", "stateId"]]

# create list of nested dictionaries 
countyStateEdges = countyStateEdges.to_dict("records")

# convert to required format
countyStateEdges = [(edge["countyId"], {}, edge["stateId"]) for edge in countyStateEdges]

# print first rows
countyStateEdges[:4]

[('01001', {}, '01'),
 ('01002', {}, '01'),
 ('01003', {}, '01'),
 ('01004', {}, '01')]

In [57]:
# insert into the database
create_relationships(g.auto(), countyStateEdges, "LOCATED_IN", \
    start_node_key=("County", "countyId"), end_node_key=("State", "stateId"))

## Stations

In this section the station data is transferred to the neo4j database. This includes the creation of the station nodes and the creation of the edges from the stations to the counties.

The station data is available in two Mongo DB collections.  The first collection is the  `Station ` collection. This collection contains all stations that existed in January 2020. Many of the stations in this collection are regional stations that are not used by long distance trains. However, some stations that are served by long-distance trains are not included in this collection (e.g. Lindau Reutin, which became operational in December 2020). The second collection of station data is named  `Train ` and contains the timetable of trains.  The timetable contains the columns IBNR (identification of the station), name, longitude and latitude of the stations served by long-distance trains.

We process the data in three steps. Since we do not want to have stations without long-distance trains in our database, we first extract all stations served by long-distance trains from the timetables (collection `train`). Then we add additional attributes from the `station` collection. After that, we perform various transformation steps. Finally, we add the station nodes and the edges from the stations to the counties into the neo4j graph database. 

In the next code cell, the station data (IBNR, name, latitude, longitude) is extracted from the timetable and stored in a Pandas data frame. We consider only German stations. The IBNR (station identifier) of these stations starts with the digits "80".

In [58]:
# create empty list
stationList = []

# query train timetable
result = db.stops.find({},{"stop":1,"_id":0})


for train in result:

    timetableTrain = train["stop"]

    for stop in timetableTrain:

        # create empty dict
        stationDict = {}

        # only add station to dictionary which is not yet in the list and which is in germany
        if not any(d["IBNR"] == stop["stopId"] for d in stationList) and str(stop["stopId"])[:2]=="80":

            stationDict["IBNR"] = stop["stopId"]
            stationDict["name"] = html.unescape(stop["stopName"])
            stationDict["longitude"] = float(stop["lon"].replace(',','.'))
            stationDict["latitude"] = float(stop["lat"].replace(',','.'))

            stationList.append(stationDict)

# convert to dataframe
stationDf = pd.DataFrame(stationList)

# display sample data
stationDf.head()

,IBNR,name,longitude,latitude
0,8010085,Dresden Hbf,13.732039,51.040562
1,8010089,Dresden-Neustadt,13.740704,51.065903
2,8010099,Elsterwerda,13.516432,51.459675
3,8010079,Doberlug-Kirchhain,13.564237,51.620573
4,8011201,Flughafen BER - Terminal 1-2,13.511947,52.364808


Next, we add more columns from the collection `station` to the Pandas dataframe by performing a feft join. These are the columns `Operator_No` and `IFOPT`. The `IFOPT` column contains information about the county.

In [59]:
#  query data
stationMap = pd.DataFrame(list(db.station.find({},{"_id":0})))

# rename key column
stationMap = stationMap.rename(columns={"EVA_NR":"IBNR"})

# select relevant data
stationMap = stationMap[["IBNR", "IFOPT", "Betreiber_Nr"]]

# join to existing data frame
stationDf = pd.merge(stationDf, stationMap, how="left", on="IBNR")

# display top rows
stationDf.head()

,IBNR,name,longitude,latitude,IFOPT,Betreiber_Nr
0,8010085,Dresden Hbf,13.732039,51.040562,de:14612:28,1343.0
1,8010089,Dresden-Neustadt,13.740704,51.065903,de:14612:16,1352.0
2,8010099,Elsterwerda,13.516432,51.459675,de:12062:900415502,1569.0
3,8010079,Doberlug-Kirchhain,13.564237,51.620573,de:12062:900415112,1240.0
4,8011201,Flughafen BER - Terminal 1-2,13.511947,52.364808,de:12061:900260009,NaN


The timetable data uses the IBNR to identify the stations. Large stations can have several IBNR. Examples are Berlin Hbf or Munich Hbf.

In [28]:
stationDf[stationDf["name"].str.contains("München Hbf")]

,IBNR,name,longitude,latitude,IFOPT,Betreiber_Nr
109,8000261,München Hbf,11.558339,48.140229,de:09162:100,4234.0
317,8098261,München Hbf Gl.27-36,11.556712,48.141532,de:09162:100:27,4234.0
347,8098262,München Hbf Gl.5-10,11.555373,48.140094,de:09162:100,4234.0


One task of the project is to find the shortest train connections between two cities with their respective interchange stations. This is not possible when using IBNR as a station key, because in this case each station part of some large stations (like Munich Hbf) is considered as a single station and trains arrive and depart at different station parts. To solve this problem, we need to combine/connect all parts of a station.

For this purpose we define a new station key. Most stations (all stations with several parts) have a so-called `Betreiber_Nr`. This number is the same for all parts of a station. However, for some stations the `Betreiber_Nr` is not available. Therefore our own station key is defined as follows: `Betreiber_Nr` if available, otherwise `IBNR`. The new station key has a letter as prefix. The If `Betreiber_Nr` is used, "B" is used as prefix, otherwise "I" is used as prefix.

The `stationKey` is created in the next cell. Also, the countyId is extracted from the `IFOPT` column.

In [60]:
# add prefix B to col "Betreiber_Nr"
stationDf["Betreiber_Nr"] =  ('B'+stationDf["Betreiber_Nr"].astype("Int64").astype("string"))

# generate station key
stationDf["stationKey"] = stationDf.Betreiber_Nr.combine_first(('I'+stationDf["IBNR"].astype("string")))

# extract countyId from IFOPT column (countyId is in the middle)
stationDf["countyId"] = stationDf["IFOPT"].str.split(":",expand=True)[1]

# display first rows
stationDf.head()

,IBNR,name,longitude,latitude,IFOPT,Betreiber_Nr,stationKey,countyId
0,8010085,Dresden Hbf,13.732039,51.040562,de:14612:28,B1343,B1343,14612
1,8010089,Dresden-Neustadt,13.740704,51.065903,de:14612:16,B1352,B1352,14612
2,8010099,Elsterwerda,13.516432,51.459675,de:12062:900415502,B1569,B1569,12062
3,8010079,Doberlug-Kirchhain,13.564237,51.620573,de:12062:900415112,B1240,B1240,12062
4,8011201,Flughafen BER - Terminal 1-2,13.511947,52.364808,de:12061:900260009,<NA>,I8011201,12061


In [61]:
# drop unneeded columns
stationDf = stationDf.drop(["Betreiber_Nr","IFOPT"], axis=1)

Then the station nodes are created in the graph database. Stations with multiple parts have multiple lines in the data frame (example Berlin Hbf). To avoid duplicates in the database, we first need to determine which row we want to use to create a node. In this case we take the row with the smallest IBNR number. We take all other columns of the `stationDf` data frame as node attributes except `countyId`, because the connection with the counties is realized via an edge.

In [62]:
# define relevant station (minimum IBNR)
minIBNR = stationDf.groupby("stationKey", as_index=False).min("IBNR")["IBNR"]

# only select relevant stations
stationNodes = stationDf[stationDf["IBNR"].isin(minIBNR)]

# drop countyId column
stationNodes = stationNodes.drop(columns=["countyId"])

# convert to required format 
stationNodes = stationNodes.to_dict("records")

# insert state nodes --> assign label "Station"
create_nodes(g.auto(), stationNodes, labels={"Station"})

Finally, the edges from the station nodes to the county nodes are created (label `LOCATED_IN`).

In [63]:
# extract key attributes
stationCountyEdges = stationDf[["stationKey", "countyId"]]

# convert to required format
stationCountyEdges = stationCountyEdges.to_dict("records")
stationCountyEdges = [(edge["stationKey"], {}, edge["countyId"]) for edge in stationCountyEdges]

# insert relationship
create_relationships(g.auto(), stationCountyEdges, "LOCATED_IN", \
    start_node_key=("Station", "stationKey"), end_node_key=("County", "countyId"))

The following database query checks wheather each station is assigned to an county (data validation).

In [64]:
query = "MATCH (s:Station) WHERE NOT (s)-[:LOCATED_IN]->() RETURN s.stationKey AS stationKey, s.name AS stationName"
queryResult = g.run(query)
pd.DataFrame(queryResult)

,0,1
0,B2218,Göttingen
1,I8000026,Basel Bad Bf
2,I8003693,Lindau-Reutin
3,I8099503,Hildesheim Gbf


The result shows that some stations are not assigned to any county. The correct countyId was not available for these stations. Therefore, we manually determined the countyId for these stations and then created the edges.

In [65]:
# manually create connections
stationCountyEdges = []
stationCountyEdges.append(("B2218", {}, "03159"))
stationCountyEdges.append(("I8000026", {}, "08336"))
stationCountyEdges.append(("I8003693", {}, "09776"))
stationCountyEdges.append(("I8099503", {}, "03254"))

# insert data
create_relationships(g.auto(), stationCountyEdges, "LOCATED_IN", \
    start_node_key=("Station", "stationKey"), end_node_key=("County", "countyId"))

In [66]:
# perform validation
query = "MATCH (s:Station) WHERE NOT (s)-[:LOCATED_IN]->() RETURN s.stationKey AS stationKey, s.name AS stationName"
queryResult = g.run(query)
pd.DataFrame(queryResult)

""


The validation shows that all stations are now assigned to a county.

## Trains

In the last section, the trains are transferred to the graph database. The trains are mapped as edges between two station nodes (label `TRAIN`).

In the next code cell, the data is first prepared (desired format) to be inserted into the graph database. This requires several transformation steps. The original schedule contains the IBNR as the station identifier. However, we use our own station key, as described in the previous section. Therefore, the station identifier attributes need to be changed (keemapping). Also, various attributes (e.g. on-board restaurant, bicycle) have to be extracted and the travel time between the two stations has to be calculated (arrTime - depTime).

In [67]:
# get keymapping
keymapping =  stationDf[["IBNR", "stationKey"]].to_dict("records")

# create empty list
connections = []

# query train timetable
result = db.stops.find({},{"stop":1,"_id":0})


for train in result:

    timetableTrain = train["stop"]

    for stopNr in range(len(timetableTrain)-1):


        # only consider lags in germany --> The IBNR number of german stations begin with 80
        if str(timetableTrain[stopNr]["stopId"])[:2] != "80" or str(timetableTrain[stopNr+1]["stopId"])[:2] != "80":
            continue
 
        # get departure stop key
        depIndex = next((index for (index, d) in enumerate(keymapping) if d["IBNR"] == timetableTrain[stopNr]["stopId"]), -1)
        depKey = keymapping[depIndex]["stationKey"]

        # get arrival stop key
        arrIndex = next((index for (index, d) in enumerate(keymapping) if d["IBNR"] == timetableTrain[stopNr+1]["stopId"]), -1)
        arrKey = keymapping[arrIndex]["stationKey"]


        # only insert connection, it it is not yet in the dictionary
        if not any(depKey == d[0] and timetableTrain[stopNr]["train"] == d[1]["trainNumber"] for d in connections):

            # create empty list
            connection = []

            # departure station
            # ------------------------
            connection.append(depKey)

            # relationship attributes
            # ------------------------

            # base attributes
            rel = {
                "trainNumber":timetableTrain[stopNr]["train"],
                "trainType":timetableTrain[stopNr]["type"],
                "depTime":timetableTrain[stopNr]["depTime"],
                "arrTime":timetableTrain[stopNr+1]["arrTime"],
                "lag":stopNr,
                "bordrestaurant": (True if any(d["key"]=="BR" for d in timetableTrain[stopNr]["notes"])             # check train ICE 205
                                        and  any(d["key"]=="BR" for d in timetableTrain[stopNr+1]["notes"]) else False),
                "bordbistro": (True if any(d["key"]=="BT" for d in timetableTrain[stopNr]["notes"])
                                        and  any(d["key"]=="BT" for d in timetableTrain[stopNr+1]["notes"]) else False),
                "bicycle": (True if any(d["key"]=="FR" for d in timetableTrain[stopNr]["notes"])
                                        and  any(d["key"]=="FR" for d in timetableTrain[stopNr+1]["notes"]) else False)
            }             

            # calculate and add duration of lag in minutes
            duration  = datetime.datetime.strptime(rel["arrTime"], '%H:%M') - datetime.datetime.strptime(rel["depTime"], '%H:%M')

            if duration.days < 0:   # trains running over midnight 
                    duration = timedelta(days=0,seconds=duration.seconds)

            rel["duration"] = int(duration.seconds/60)

            connection.append(rel) 

            # arrival station
            # ------------------------
            connection.append(arrKey)

            # convert to tuple and append to connection list
            connections.append(tuple(connection))

# print connections
connections[:2]

[('B1343',
  {'trainNumber': 'IC 2274',
   'trainType': 'IC',
   'depTime': '05:19',
   'arrTime': '05:24',
   'lag': 0,
   'bordrestaurant': False,
   'bordbistro': False,
   'bicycle': True,
   'duration': 5},
  'B1352'),
 ('B1352',
  {'trainNumber': 'IC 2274',
   'trainType': 'IC',
   'depTime': '05:26',
   'arrTime': '05:54',
   'lag': 1,
   'bordrestaurant': False,
   'bordbistro': False,
   'bicycle': True,
   'duration': 28},
  'B1569')]

Then the edges are inserted into the database.

In [68]:
create_relationships(g.auto(), connections, "TRAIN", start_node_key=("Station", "stationKey"), end_node_key=("Station", "stationKey"))

If all data has been entered correctly, each train must be displayed as a continuous chain (no breaks). The validation query returns all trains where this is not the case.

In [69]:
query = """MATCH () -[t:TRAIN]-> ()
           WITH t.trainNumber as name, COUNT(t) as actual, MAX(t.lag)-MIN(t.lag)+1 as correct
           WHERE actual <> correct
           RETURN name, actual, correct"""

pd.DataFrame(g.run(query), columns=["trainNumber", "actual", "correct"])

,trainNumber,actual,correct


Since the result set is empty, the trains are correctly represented in the database.